In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=101d88ffb7122b744cf74e1769aaca05920421b18784cd00a1952d1b7af164f0
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 1496.92 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_2_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_2_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=856727, Mon Apr 13 13:55:12 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.489019303013667e+07 1.0e+00 2.54e-01  3e-01  3e-01 0:00.0
    2     30 9.472948513928953e+07 1.0e+00 2.44e-01  2e-01  2e-01 0:00.0
    3     45 9.469733342399512e+07 1.0e+00 2.40e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [94896913.45061482] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [94891094.17103398] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -6.928291898683465e+07 1.4e+00 3.43e-02  3e-02  3e-02 0:01.9
  200   3000 -7.403579999073052e+07 1.5e+00 2.24e-03  2e-03  2e-03 0:03.9
  300   4500 -7.405544250339231e+07 1.5e+00 1.16e-04  1e-04  1e-04 0:05.8
  400   6000 -7.405551409825775e+07 1.6e+00 7.89e-06  6e-06  7e-06 0:07.8
  500   7500 -7.405551433056536e+07 1.6e+00 4.02e-07  3e-07  4e-07 0:09.7
  600   9000 -7.405551433123556e+07 1.7e+00 2.64e-08  2e-08  2e-08 0:11.6
  700  10500 -7.405551433123806e+07 1.7e+00 2.42e-09  2e-09  2e-09 0:13.6
  800  12000 -7.405551433123806e+07 1.8e+00 1.35e-09  1e-09  1e-09 0:15.5
  900  13500 -7.405551433123809e+07 2.0e+00 6.62e-10  5e-10  6e-10 0:17.4
 1000  15000 -7.405551433123809e+07 2.3e+00 7.29e-10  5e-10  6e-10 0:19.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -7.405551433123806e+07 2.8e+00 5.81e-10  4e-10  5e-10 0:21.3
 1200  18000 -7.405551433123806e+07 2.9e+00 5.35e-10  4e-10  5e-10 0:23.2
 1300  19500 -7.405551433123806e+07 3.1e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [92935825.15909508] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 -6.728649134559442e+14 1.5e+00 2.55e+02  2e+02  3e+02 0:01.8
  101   1515 -8.058979140447305e+14 1.5e+00 2.75e+02  3e+02  3e+02 0:01.9
termination on tolfacupx=1000.0 (Mon Apr 13 13:56:17 2020)
final/bestever f-value = -7.441869e+14 -8.058979e+14
incumbent solution: [  770.47112618  -233.08059967  -786.46293442 -1005.36347381
   -88.54519153   322.14675237     4.85331196   589.8627582  ...]
std deviations: [273.23671393 270.1686317  263.92348277 265.9948116  264.66701882
 262.11746829 272.16463214 264.68690546 ...]
Run : 1
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=958948, Mon Apr 13 13:56:17 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.455993867415908e+07 1.0e+00 2.56e-01  3e-01  3e-01 0:00.0
    2     30 9.431330280828425e+07 1.0e+00 2.48e-01  2e-01  2e-01 0:00.0
    3     45 9.408458338223191e+07 1.1e+00 2.42e-01  2e-01  2e-01 0:00.1


...

  100   1500 -1.802205709847992e+14 1.4e+00 1.47e+02  1e+02  1e+02 0:01.8
  109   1635 -7.521058760090095e+14 1.4e+00 2.84e+02  3e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 14:27:31 2020)
final/bestever f-value = -7.376180e+14 -7.521059e+14
incumbent solution: [ 393.01992927 -421.48378615 -256.24510018 -687.89921842  551.03799975
  434.80796855  260.6613231  1043.80387069 ...]
std deviations: [269.52893926 275.57289635 277.03738468 277.1704129  272.17896761
 268.34754604 274.32691032 271.41153946 ...]
Run : 29
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=858962, Mon Apr 13 14:27:31 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.486321613688423e+07 1.0e+00 2.54e-01  3e-01  3e-01 0:00.0
    2     30 9.482274856288713e+07 1.0e+00 2.45e-01  2e-01  2e-01 0:00.0
    3     45 9.473348442214936e+07 1.0e+00 2.42e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [94919543.20816016] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [94905619.09180126] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -6.910225204898140e+07 1.4e+00 2.73e-02  3e-02  3e-02 0:01.9
  200   3000 -7.404864660040751e+07 1.5e+00 1.35e-03  1e-03  1e-03 0:03.8
  300   4500 -7.405549143915990e+07 1.5e+00 6.70e-05  6e-05  6e-05 0:05.7
  400   6000 -7.405551426171455e+07 1.6e+00 3.73e-06  3e-06  3e-06 0:07.6
  500   7500 -7.405551433096126e+07 1.6e+00 2.45e-07  2e-07  2e-07 0:09.5
  600   9000 -7.405551433123758e+07 1.7e+00 1.22e-08  9e-09  1e-08 0:11.4
  700  10500 -7.405551433123806e+07 1.8e+00 1.79e-09  1e-09  2e-09 0:13.2
  800  12000 -7.405551433123806e+07 2.1e+00 1.67e-09  1e-09  1e-09 0:15.1
  900  13500 -7.405551433123812e+07 2.4e+00 1.32e-09  9e-10  1e-09 0:17.0
 1000  15000 -7.405551433123806e+07 2.5e+00 9.56e-10  7e-10  9e-10 0:18.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -7.405551433123806e+07 2.6e+00 6.34e-10  4e-10  6e-10 0:20.8
 1200  18000 -7.405551433123812e+07 2.8e+00 5.78e-10  4e-10  5e-10 0:22.7
 1300  19500 -7.405551433123803e+07 3.0e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [92935825.15909508] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 -1.776170049022829e+14 1.6e+00 1.32e+02  1e+02  1e+02 0:01.9
  110   1650 -7.079209580304212e+14 1.6e+00 2.57e+02  2e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 14:28:32 2020)
final/bestever f-value = -6.699452e+14 -7.079210e+14
incumbent solution: [ 349.17709953   27.76448182  205.28214375 -801.50577734  156.34404046
   10.27977353  730.52255554  467.542525   ...]
std deviations: [248.48768743 253.18094366 249.34649253 248.55364824 246.68637849
 244.54767204 246.54935324 246.67191185 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F2_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:50])
    SVM_Fun [i] = F2(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F2(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([24744031.86046319]), count=array([1])) 0.036556112188563666
SVM
ModeResult(mode=array([14295743.39322072]), count=array([1])) 2776588.0961683304
ELN
ModeResult(mode=array([1.73112603e+12]), count=array([1])) 1605187953881.689
